### Combine the SoupX corrected count matrices and save as adata

In [1]:
# Load required libraries
library(reticulate)
library(Matrix)

In [11]:
# directories for each of the samples
sample_dirs <- list.dirs("../raw_data_Dropseq/", recursive=FALSE)
sample_dirs

[1] "../raw_data_Dropseq//Fetal_LV_18wk_e1-run1n2"    
 [2] "../raw_data_Dropseq//Fetal-1st-LV-0315-1-run1n2" 
 [3] "../raw_data_Dropseq//Fetal-1st-LV-0315-2-run1n2" 
 [4] "../raw_data_Dropseq//Fetal-2nd-Atria-1"          
 [5] "../raw_data_Dropseq//Fetal-2nd-Atria-2"          
 [6] "../raw_data_Dropseq//Fetal-2nd-OFT-1"            
 [7] "../raw_data_Dropseq//Fetal-2nd-OFT-2"            
 [8] "../raw_data_Dropseq//Fetal-Atria-18wk"           
 [9] "../raw_data_Dropseq//Fetal-LRV-18wk-male1-run1n2"
[10] "../raw_data_Dropseq//Fetal-LRV-18wk-male2-run1n2"
[11] "../raw_data_Dropseq//Fetal-LRV-18wk-male3-run1n2"
[12] "../raw_data_Dropseq//Fetal-LRV-18wk-male4-run1n2"
[13] "../raw_data_Dropseq//Fetal-LV-18wk-e2-run1n2"    
[14] "../raw_data_Dropseq//Fetal-LV-18wk-e3"

In [12]:
# extract the names of each of the samples
sample_names <- basename(sample_dirs)

In [13]:
# extract the paths to the SoupX corrected matrices
SoupX_count_rds_paths <- paste0(sample_dirs, "/STAR/Solo.out/GeneFullsoupX_corrected_counts.rds")

In [14]:
# keep only those for which there was a stable contamination estimation (this drops one of the files)
SoupX_count_rds_paths <- SoupX_count_rds_paths[file.exists(SoupX_count_rds_paths)]

In [15]:
output_adata_dir <- "adata_from_SoupX/"
dir.create(output_adata_dir)

Warning message in dir.create(output_adata_dir):
“'adata_from_SoupX' already exists”


Iterate through each of the SoupX rds files and create a corresponding adata file in the output_adata_dir

In [16]:
num_SoupX_rds <- length(SoupX_count_rds_paths)

for (i in 1:num_SoupX_rds) {

    if (i %% 5 == 0) {
        print(i)
        flush.console()
    }
     
    sample_name <- sample_names[i]

    # load in the SoupX directory
    SoupX_count <- readRDS(SoupX_count_rds_paths[i])
    # transpose for adata format since the rows should correspond to the cells and the columns to the genes
    sparse_matrix <- t(SoupX_count)
    # Import the anndata module from Python
    anndata <- import("anndata")
    # Create an adata object, with the adata.obs as the rownames of the transposed sparse matrix 
    # and adata.var as the colnames
    adata <- anndata$AnnData(X = sparse_matrix, 
                            obs = data.frame(row.names = rownames(sparse_matrix)), 
                            var = data.frame(row.names = colnames(sparse_matrix)))
    
    # save the adata file
    output_adata_path <- paste0(output_adata_dir, sample_name, ".h5ad")
    adata$write(output_adata_path)
}

[1] 5
[1] 10
